# Calories Prediction - Neural Network Model

This notebook implements a Neural Network model to predict calories burned during workouts for the Kaggle Playground Series competition.


## 1. Import Libraries


In [2]:
# Data manipulation
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_log_error, mean_squared_error, r2_score

# Deep learning
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import multiprocessing

# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42) if torch.cuda.is_available() else None

# Set device - check for CUDA first, then MPS (for Mac M1/M2), then fall back to CPU
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS (Metal Performance Shaders) device")
else:
    device = torch.device("cpu")
    print("Using CPU device")

# Set up multiprocessing for CPU if needed
num_workers = 0
if device.type == 'cpu':
    # Set multiprocessing start method (for Jupyter notebook compatibility)
    try:
        multiprocessing.set_start_method('spawn', force=True)
        print("Using 'spawn' start method for multiprocessing")
    except RuntimeError:
        print("Could not set start method to 'spawn' for multiprocessing")

    # Set number of workers for DataLoader
    num_workers = multiprocessing.cpu_count() - 1  # Leave one CPU for system tasks
    num_workers = max(0, num_workers)  # Ensure non-negative

    # Enable multiprocessing for PyTorch
    torch.set_num_threads(num_workers)
    print(f"Using {num_workers} workers for data loading and {torch.get_num_threads()} threads for PyTorch operations")


Using MPS (Metal Performance Shaders) device


## 2. Load and Explore Data


In [3]:
# Load the training and test data
train_data = pd.read_csv('playground-series-s5e5/train.csv')
test_data = pd.read_csv('playground-series-s5e5/test.csv')

# Display basic information about the training data
print("Training data shape:", train_data.shape)
train_data.head()


Training data shape: (750000, 9)


,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0


In [4]:
# Check for missing values
print("Missing values in training data:")
print(train_data.isnull().sum())

print("\nMissing values in test data:")
print(test_data.isnull().sum())


Missing values in training data:
id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

Missing values in test data:
id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
dtype: int64


In [5]:
# Statistical summary of the training data
train_data.describe()


,id,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
count,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000,750000.000000
mean,374999.500000,41.420404,174.697685,75.145668,15.421015,95.483995,40.036253,88.282781
std,216506.495284,15.175049,12.824496,13.982704,8.354095,9.449845,0.779875,62.395349
min,0.000000,20.000000,126.000000,36.000000,1.000000,67.000000,37.100000,1.000000
25%,187499.750000,28.000000,164.000000,63.000000,8.000000,88.000000,39.600000,34.000000
50%,374999.500000,40.000000,174.000000,74.000000,15.000000,95.000000,40.300000,77.000000
75%,562499.250000,52.000000,185.000000,87.000000,23.000000,103.000000,40.700000,136.000000
max,749999.000000,79.000000,222.000000,132.000000,30.000000,128.000000,41.500000,314.000000


## 3. Feature Engineering


In [6]:
# Create a copy of the datasets for feature engineering
train_fe = train_data.copy()
test_fe = test_data.copy()

# Calculate BMI (Body Mass Index)
train_fe['BMI'] = train_fe['Weight'] / ((train_fe['Height'] / 100) ** 2)
test_fe['BMI'] = test_fe['Weight'] / ((test_fe['Height'] / 100) ** 2)

# Create interaction features
train_fe['Duration_HeartRate'] = train_fe['Duration'] * train_fe['Heart_Rate']
test_fe['Duration_HeartRate'] = test_fe['Duration'] * test_fe['Heart_Rate']

train_fe['Weight_Duration'] = train_fe['Weight'] * train_fe['Duration']
test_fe['Weight_Duration'] = test_fe['Weight'] * test_fe['Duration']

# Convert Sex to numerical (0 for female, 1 for male)
train_fe['Sex_num'] = train_fe['Sex'].map({'female': 0, 'male': 1})
test_fe['Sex_num'] = test_fe['Sex'].map({'female': 0, 'male': 1})

# Display the new features
train_fe.head()


,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories,BMI,Duration_HeartRate,Weight_Duration,Sex_num
0,0,male,36,189.0,82.0,26.0,101.0,41.0,150.0,22.955684,2626.0,2132.0,1
1,1,female,64,163.0,60.0,8.0,85.0,39.7,34.0,22.582709,680.0,480.0,0
2,2,female,51,161.0,64.0,7.0,84.0,39.8,29.0,24.690405,588.0,448.0,0
3,3,male,20,192.0,90.0,25.0,105.0,40.7,140.0,24.414062,2625.0,2250.0,1
4,4,female,38,166.0,61.0,25.0,102.0,40.6,146.0,22.136740,2550.0,1525.0,0


## 4. Prepare Data for Modeling


In [7]:
# Define features and target
features = ['Age', 'Height', 'Weight', 'Duration', 'Heart_Rate', 'Body_Temp', 'Sex_num', 'BMI', 'Duration_HeartRate', 'Weight_Duration']
X = train_fe[features]
y = train_fe['Calories']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Validation set shape: {X_val.shape}")


Training set shape: (600000, 10)
Validation set shape: (150000, 10)


In [8]:
# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Prepare test data
X_test = test_fe[features]
X_test_scaled = scaler.transform(X_test)


## 5. Build a Basic Neural Network Model


In [9]:
# Define a PyTorch neural network model
class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, hidden_layers=[64, 32], activation='relu', 
                 dropout_rate=0.2, l1_reg=0.0, l2_reg=0.0):
        super(NeuralNetwork, self).__init__()

        # Set activation function
        if activation == 'relu':
            self.activation = nn.ReLU()
        elif activation == 'elu':
            self.activation = nn.ELU()
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        else:
            self.activation = nn.ReLU()  # Default to ReLU

        # Create layers list
        layers = []

        # Input layer
        layers.append(nn.Linear(input_dim, hidden_layers[0]))
        layers.append(self.activation)
        layers.append(nn.Dropout(dropout_rate))

        # Hidden layers
        for i in range(len(hidden_layers) - 1):
            layers.append(nn.Linear(hidden_layers[i], hidden_layers[i+1]))
            layers.append(self.activation)
            layers.append(nn.Dropout(dropout_rate))

        # Output layer
        layers.append(nn.Linear(hidden_layers[-1], 1))

        # Create sequential model
        self.model = nn.Sequential(*layers)

        # Store regularization parameters
        self.l1_reg = l1_reg
        self.l2_reg = l2_reg

    def forward(self, x):
        return self.model(x)

    def l1_regularization(self):
        l1_loss = 0
        for param in self.parameters():
            l1_loss += torch.sum(torch.abs(param))
        return self.l1_reg * l1_loss

    def l2_regularization(self):
        l2_loss = 0
        for param in self.parameters():
            l2_loss += torch.sum(param.pow(2))
        return self.l2_reg * l2_loss

# Function to create and configure the model
def create_model(input_dim, hidden_layers=[64, 32], activation='relu', 
                 learning_rate=0.001, dropout_rate=0.2, l1_reg=0.0, l2_reg=0.0):
    # Create model
    model = NeuralNetwork(
        input_dim=input_dim,
        hidden_layers=hidden_layers,
        activation=activation,
        dropout_rate=dropout_rate,
        l1_reg=l1_reg,
        l2_reg=l2_reg
    ).to(device)

    # Define optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    return model, optimizer


In [10]:
# Convert data to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train.values).to(device).reshape(-1, 1)
X_val_tensor = torch.FloatTensor(X_val_scaled).to(device)
y_val_tensor = torch.FloatTensor(y_val.values).to(device).reshape(-1, 1)

# Create a basic neural network model
input_dim = X_train_scaled.shape[1]
basic_model, basic_optimizer = create_model(input_dim)

# Display model summary
print(basic_model)
print(f"Total parameters: {sum(p.numel() for p in basic_model.parameters())}")


NeuralNetwork(
  (activation): ReLU()
  (model): Sequential(
    (0): Linear(in_features=10, out_features=64, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=64, out_features=32, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=32, out_features=1, bias=True)
  )
)
Total parameters: 2817


In [ ]:
# Define a function for mean squared logarithmic error
def msle_loss(pred, target):
    # Ensure predictions are positive
    pred = torch.clamp(pred, min=1e-6)
    target = torch.clamp(target, min=1e-6)

    # Calculate MSLE
    return torch.mean(torch.pow(torch.log1p(pred) - torch.log1p(target), 2))

# Define a function for training with early stopping
def train_model(model, optimizer, X_train, y_train, X_val, y_val, 
                epochs=100, batch_size=32, patience=20, factor=0.2, min_lr=0.0001, 
                verbose=1):
    """
    Train a PyTorch model with early stopping and learning rate reduction.

    Parameters:
    - model: PyTorch model to train
    - optimizer: PyTorch optimizer
    - X_train, y_train: Training data
    - X_val, y_val: Validation data
    - epochs: Maximum number of epochs to train
    - batch_size: Batch size for training
    - patience: Number of epochs with no improvement after which training will be stopped
    - factor: Factor by which the learning rate will be reduced
    - min_lr: Minimum learning rate
    - verbose: Verbosity level (0=silent, 1=normal, 2=detailed)

    Returns:
    - model: Trained model
    - history: Training history
    """
    import time
    from datetime import timedelta
    import numpy as np

    # Initialize variables for early stopping and learning rate reduction
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_state = None
    train_losses = []
    val_losses = []
    train_mses = []
    val_mses = []

    # Time tracking
    start_time = time.time()
    epoch_times = []

    # Print training information
    if verbose > 0:
        print(f"\n{'='*80}")
        print(f"Starting training with {epochs} epochs, batch size {batch_size}")
        print(f"Device: {device}, Model parameters: {sum(p.numel() for p in model.parameters())}")
        print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")
        print(f"Learning rate: {optimizer.param_groups[0]['lr']}, Patience: {patience}")
        print(f"{'='*80}\n")

    # Create data loaders
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(
        train_dataset, 
        batch_size=batch_size, 
        shuffle=True,
        num_workers=num_workers if device.type == 'cpu' else 0,
        pin_memory=(device.type != 'cpu')
    )

    # Training loop
    for epoch in range(epochs):
        epoch_start_time = time.time()

        # Training phase
        model.train()
        train_loss = 0
        train_mse = 0

        # Batch progress tracking
        if verbose > 1:
            print(f"\nEpoch {epoch+1}/{epochs}:")
            print(f"{'='*50}")

        # Track batch progress
        total_batches = len(train_loader)

        for batch_idx, (X_batch, y_batch) in enumerate(train_loader):
            # Forward pass
            y_pred = model(X_batch)

            # Calculate loss
            loss = msle_loss(y_pred, y_batch)

            # Add regularization if needed
            if model.l1_reg > 0:
                l1_loss = model.l1_regularization()
                loss += l1_loss
            if model.l2_reg > 0:
                l2_loss = model.l2_regularization()
                loss += l2_loss

            # Calculate MSE for tracking
            mse = torch.mean(torch.pow(y_pred - y_batch, 2))

            # Backward pass and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accumulate batch loss
            train_loss += loss.item() * X_batch.size(0)
            train_mse += mse.item() * X_batch.size(0)

            # Print batch progress
            if verbose > 1 and (batch_idx % max(1, total_batches // 10) == 0 or batch_idx == total_batches - 1):
                batch_loss = loss.item()
                batch_mse = mse.item()
                progress = (batch_idx + 1) / total_batches * 100
                print(f"Batch {batch_idx+1}/{total_batches} [{progress:.1f}%] - Loss: {batch_loss:.6f}, MSE: {batch_mse:.6f}")

        # Calculate average training loss
        train_loss /= len(train_loader.dataset)
        train_mse /= len(train_loader.dataset)
        train_losses.append(train_loss)
        train_mses.append(train_mse)

        # Validation phase
        model.eval()
        with torch.no_grad():
            y_val_pred = model(X_val)
            val_loss = msle_loss(y_val_pred, y_val).item()
            val_mse = torch.mean(torch.pow(y_val_pred - y_val, 2)).item()

            # Calculate RMSLE for more detailed reporting
            y_val_np = y_val.cpu().numpy()
            y_val_pred_np = y_val_pred.cpu().numpy().flatten()
            y_val_pred_np = np.maximum(y_val_pred_np, 0)  # Ensure predictions are positive
            val_rmsle = np.sqrt(mean_squared_log_error(y_val_np, y_val_pred_np))

        val_losses.append(val_loss)
        val_mses.append(val_mse)

        # Calculate epoch time
        epoch_end_time = time.time()
        epoch_time = epoch_end_time - epoch_start_time
        epoch_times.append(epoch_time)

        # Calculate estimated time remaining
        avg_epoch_time = np.mean(epoch_times)
        remaining_epochs = epochs - (epoch + 1)
        estimated_time_remaining = avg_epoch_time * remaining_epochs

        # Print progress
        if verbose > 0 and (epoch % max(1, epochs // 20) == 0 or epoch == epochs - 1):
            elapsed = time.time() - start_time
            print(f"Epoch {epoch+1}/{epochs} - {timedelta(seconds=int(epoch_time))} - ETA: {timedelta(seconds=int(estimated_time_remaining))}")
            print(f"  Train Loss: {train_loss:.6f}, Train MSE: {train_mse:.6f}")
            print(f"  Val Loss: {val_loss:.6f}, Val MSE: {val_mse:.6f}, Val RMSLE: {val_rmsle:.6f}")

            # Print current learning rate
            current_lr = optimizer.param_groups[0]['lr']
            print(f"  Learning rate: {current_lr:.6f}")

            # Print improvement information
            if epoch > 0:
                prev_val_loss = val_losses[-2]
                loss_change = (prev_val_loss - val_loss) / prev_val_loss * 100
                change_sign = "↓" if loss_change > 0 else "↑"
                print(f"  Validation loss change: {change_sign} {abs(loss_change):.2f}%")

        # Early stopping
        if val_loss < best_val_loss:
            improvement = (best_val_loss - val_loss) / best_val_loss * 100
            best_val_loss = val_loss
            patience_counter = 0
            best_model_state = model.state_dict().copy()

            if verbose > 0:
                print(f"  ✓ New best validation loss: {best_val_loss:.6f} (improved by {improvement:.2f}%)")
        else:
            patience_counter += 1
            if verbose > 0:
                print(f"  ✗ No improvement for {patience_counter}/{patience} epochs. Best: {best_val_loss:.6f}")

            if patience_counter >= patience:
                print(f"\n{'='*50}")
                print(f"Early stopping at epoch {epoch+1}")
                print(f"Best validation loss: {best_val_loss:.6f}")
                print(f"{'='*50}")
                break

        # Learning rate reduction
        if patience_counter > 0 and patience_counter % 5 == 0:
            old_lr = optimizer.param_groups[0]['lr']
            for param_group in optimizer.param_groups:
                param_group['lr'] = max(param_group['lr'] * factor, min_lr)
            new_lr = optimizer.param_groups[0]['lr']

            if verbose > 0:
                print(f"  → Reducing learning rate: {old_lr:.6f} → {new_lr:.6f}")

    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    # Calculate total training time
    total_time = time.time() - start_time

    if verbose > 0:
        print(f"\n{'='*80}")
        print(f"Training completed in {timedelta(seconds=int(total_time))}")
        print(f"Best validation loss: {best_val_loss:.6f}")
        print(f"Total epochs: {len(train_losses)}/{epochs}")
        print(f"Average epoch time: {timedelta(seconds=int(np.mean(epoch_times)))}")
        print(f"{'='*80}\n")

    # Return training history
    history = {
        'loss': train_losses,
        'val_loss': val_losses,
        'mse': train_mses,
        'val_mse': val_mses,
        'epoch_times': epoch_times,
        'total_time': total_time,
        'best_val_loss': best_val_loss
    }

    return model, history

# Train the basic model
basic_model, history = train_model(
    basic_model, basic_optimizer,
    X_train_tensor, y_train_tensor,
    X_val_tensor, y_val_tensor,
    epochs=100,
    batch_size=1000,
    verbose=2  # Set to 2 for more detailed output
)



Starting training with 100 epochs, batch size 1000
Device: mps, Model parameters: 2817
Training samples: 600000, Validation samples: 150000
Learning rate: 0.001, Patience: 20


Epoch 1/100:
Batch 1/600 [0.2%] - Loss: 0.019510, MSE: 230.345383
Batch 61/600 [10.2%] - Loss: 0.021878, MSE: 263.829834
Batch 121/600 [20.2%] - Loss: 0.018556, MSE: 237.502106
Batch 181/600 [30.2%] - Loss: 0.021451, MSE: 244.836761
Batch 241/600 [40.2%] - Loss: 0.022569, MSE: 232.421295
Batch 301/600 [50.2%] - Loss: 0.030044, MSE: 216.616135
Batch 361/600 [60.2%] - Loss: 0.022076, MSE: 273.329132
Batch 421/600 [70.2%] - Loss: 0.020186, MSE: 226.174316
Batch 481/600 [80.2%] - Loss: 0.019562, MSE: 254.799820
Batch 541/600 [90.2%] - Loss: 0.019227, MSE: 227.667755
Batch 600/600 [100.0%] - Loss: 0.017921, MSE: 218.830795
Epoch 1/100 - 0:00:12 - ETA: 0:19:50
  Train Loss: 0.020995, Train MSE: 234.900398
  Val Loss: 0.004555, Val MSE: 17.878750, Val RMSLE: 0.067491
  Learning rate: 0.001000
  ✓ New best validation l

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['mse'], label='Training MSE')
plt.plot(history['val_mse'], label='Validation MSE')
plt.title('Mean Squared Error')
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
# Make predictions on the validation set
basic_model.eval()
with torch.no_grad():
    y_pred_basic = basic_model(X_val_tensor).cpu().numpy().flatten()

# Ensure predictions are positive (required for log calculation)
y_pred_basic = np.maximum(y_pred_basic, 0)

# Calculate RMSLE (Root Mean Squared Logarithmic Error)
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# Calculate metrics
rmsle_score = rmsle(y_val, y_pred_basic)
rmse_score = np.sqrt(mean_squared_error(y_val, y_pred_basic))
r2 = r2_score(y_val, y_pred_basic)

print(f"Basic Neural Network Model Performance:")
print(f"RMSLE: {rmsle_score:.4f}")
print(f"RMSE: {rmse_score:.4f}")
print(f"R² Score: {r2:.4f}")


## 6. Hyperparameter Tuning


In [ ]:
# Define a function to evaluate a model with given hyperparameters
def evaluate_model(hidden_layers, activation, learning_rate, dropout_rate, l1_reg, l2_reg, batch_size):
    # Create the model and optimizer
    model, optimizer = create_model(
        input_dim=input_dim,
        hidden_layers=hidden_layers,
        activation=activation,
        learning_rate=learning_rate,
        dropout_rate=dropout_rate,
        l1_reg=l1_reg,
        l2_reg=l2_reg
    )

    # Train the model
    model, history = train_model(
        model, optimizer,
        X_train_tensor, y_train_tensor,
        X_val_tensor, y_val_tensor,
        epochs=100,
        batch_size=batch_size,
        patience=20,
        factor=0.2,
        min_lr=0.0001,
        verbose=1  # Set to 2 for more detailed output
    )

    # Evaluate the model
    model.eval()
    with torch.no_grad():
        y_pred = model(X_val_tensor).cpu().numpy().flatten()

    y_pred = np.maximum(y_pred, 0)  # Ensure predictions are positive
    rmsle_val = rmsle(y_val, y_pred)

    return model, rmsle_val, history


In [ ]:
# Define hyperparameter combinations to try
hyperparameter_combinations = [
    # Hidden layers, activation, learning rate, dropout rate, l1 reg, l2 reg, batch size
    [[128, 64], 'relu', 0.001, 0.2, 0.0, 0.0, 32],
    [[256, 128, 64], 'relu', 0.001, 0.3, 0.0, 0.001, 32],
    [[128, 64, 32], 'elu', 0.0005, 0.2, 0.0, 0.001, 64],
    [[64, 32], 'relu', 0.001, 0.1, 0.0001, 0.0, 32],
    [[128, 64, 32, 16], 'relu', 0.001, 0.2, 0.0, 0.001, 32]
]

# Evaluate each combination
results = []
for i, params in enumerate(hyperparameter_combinations):
    print(f"Training model {i+1}/{len(hyperparameter_combinations)}...")
    model, rmsle_val, history = evaluate_model(*params)
    results.append({
        'model': model,
        'params': params,
        'rmsle': rmsle_val,
        'history': history
    })
    print(f"RMSLE: {rmsle_val:.4f}")

# Find the best model
best_result = min(results, key=lambda x: x['rmsle'])
print("\nBest model parameters:")
print(f"Hidden layers: {best_result['params'][0]}")
print(f"Activation: {best_result['params'][1]}")
print(f"Learning rate: {best_result['params'][2]}")
print(f"Dropout rate: {best_result['params'][3]}")
print(f"L1 regularization: {best_result['params'][4]}")
print(f"L2 regularization: {best_result['params'][5]}")
print(f"Batch size: {best_result['params'][6]}")
print(f"RMSLE: {best_result['rmsle']:.4f}")


In [ ]:
# Get the best model
best_model = best_result['model']

# Make predictions on the validation set
best_model.eval()
with torch.no_grad():
    y_pred_best = best_model(X_val_tensor).cpu().numpy().flatten()

y_pred_best = np.maximum(y_pred_best, 0)  # Ensure predictions are positive

# Calculate metrics
rmsle_score_best = rmsle(y_val, y_pred_best)
rmse_score_best = np.sqrt(mean_squared_error(y_val, y_pred_best))
r2_best = r2_score(y_val, y_pred_best)

print(f"Best Neural Network Model Performance:")
print(f"RMSLE: {rmsle_score_best:.4f}")
print(f"RMSE: {rmse_score_best:.4f}")
print(f"R² Score: {r2_best:.4f}")

# Compare with the basic model
print("\nImprovement over basic model:")
print(f"RMSLE improvement: {rmsle_score - rmsle_score_best:.4f} ({(rmsle_score - rmsle_score_best) / rmsle_score * 100:.2f}%)")


## 7. Visualize Predictions vs Actual Values


In [ ]:
# Create a dataframe with actual and predicted values
results_df = pd.DataFrame({
    'Actual': y_val,
    'Predicted': y_pred_best
})

# Plot actual vs predicted values
plt.figure(figsize=(10, 8))
plt.scatter(results_df['Actual'], results_df['Predicted'], alpha=0.5)
plt.plot([min(y_val), max(y_val)], [min(y_val), max(y_val)], 'r--')
plt.xlabel('Actual Calories')
plt.ylabel('Predicted Calories')
plt.title('Actual vs Predicted Calories')
plt.show()

# Plot residuals
results_df['Residuals'] = results_df['Actual'] - results_df['Predicted']
plt.figure(figsize=(10, 6))
sns.histplot(results_df['Residuals'], kde=True)
plt.axvline(x=0, color='r', linestyle='--')
plt.title('Residuals Distribution')
plt.xlabel('Residuals')
plt.show()


## 8. Make Predictions on Test Data


In [ ]:
# Convert test data to PyTorch tensor
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)

# Make predictions on the test set
best_model.eval()
with torch.no_grad():
    test_predictions = best_model(X_test_tensor).cpu().numpy().flatten()

# Ensure predictions are positive
test_predictions = np.maximum(test_predictions, 0)

# Create submission dataframe
submission = pd.DataFrame({
    'id': test_data['id'],
    'Calories': test_predictions
})

# Display the first few rows of the submission file
submission.head()


In [ ]:
# Save the submission file
submission.to_csv('neural_network_submission.csv', index=False)
print("Submission file saved successfully!")


## 9. Feature Importance Analysis (Permutation Importance)


In [ ]:
# Since neural networks don't provide feature importance directly,
# we can use permutation importance to estimate feature importance

from sklearn.inspection import permutation_importance

# Create a wrapper for the PyTorch model to use with scikit-learn
class PyTorchRegressor:
    def __init__(self, model, device):
        self.model = model
        self.device = device
        self.model.eval()  # Set model to evaluation mode

    def predict(self, X):
        # Convert numpy array to PyTorch tensor
        X_tensor = torch.FloatTensor(X).to(self.device)

        # Make predictions
        with torch.no_grad():
            predictions = self.model(X_tensor).cpu().numpy().flatten()

        return predictions

# Create a wrapper for our best model
torch_wrapper = PyTorchRegressor(best_model, device)

# Calculate permutation importance
result = permutation_importance(
    torch_wrapper, X_val_scaled, y_val,
    n_repeats=10,
    random_state=42,
    n_jobs=-1
)

# Create a dataframe with feature importances
feature_importance = pd.DataFrame({
    'Feature': features,
    'Importance': result.importances_mean
})
feature_importance = feature_importance.sort_values('Importance', ascending=False)

# Plot feature importances
plt.figure(figsize=(12, 8))
sns.barplot(x='Importance', y='Feature', data=feature_importance)
plt.title('Feature Importance (Permutation Importance)')
plt.tight_layout()
plt.show()


## 10. Conclusion

In this notebook, we built a Neural Network model to predict calories burned during workouts. We performed the following steps:

1. Loaded and explored the dataset
2. Performed feature engineering to create new features that might improve model performance
3. Prepared the data for modeling, including scaling the features
4. Built a basic neural network model as a baseline
5. Tuned the hyperparameters of the model, including network architecture, activation functions, and regularization
6. Trained an optimized neural network model with the best hyperparameters
7. Evaluated the model's performance using RMSLE (Root Mean Squared Logarithmic Error)
8. Analyzed feature importance using permutation importance
9. Generated predictions for the test set and created a submission file

The optimized neural network model showed good performance on the validation set, with an improvement over the baseline model. The most important features for predicting calories burned were identified through permutation importance analysis.
